# Exercise - Get monthly crime count by type

- Data set URL - https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Data is available in HDFS file system under /public/crime/csv
- You can check properties of files using hadoop fs -ls -h /public/crime/csv
- Structure of data (ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location)
- File format - text file
- Delimiter - “,”
- Get monthly count of primary crime type, sorted by month in ascending and number of crimes per type in descending order
- Store the result in HDFS path /user/YOUR_USER_ID/solutions/solution01/crimes_by_type_by_month
- Output File Format: TEXT
- Output Columns: Month in YYYYMM format, crime count, crime type
- Output Delimiter: \t (tab delimited)
- Output Compression: gzip
- Validate the results

In [3]:
%%bash

hdfs dfs -ls /public/crime/

Found 2 items
drwxr-xr-x   - hdfs hdfs          0 2017-08-08 04:34 /public/crime/csv
drwxr-xr-x   - hdfs hdfs          0 2017-08-08 04:36 /public/crime/json


In [4]:
%%bash

hdfs dfs -ls -h /public/crime/csv

Found 1 items
-rw-r--r--   3 hdfs hdfs      1.4 G 2017-08-08 04:34 /public/crime/csv/crime_data.csv


In [5]:
%%bash

hdfs dfs -cat /public/crime/csv/crime_data.csv | head

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
5679862,HN487108,07/24/2007 10:11:00 PM,054XX S ABERDEEN ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,false,false,0934,009,16,61,14,1169912,1868555,2007,04/15/2016 08:55:02 AM,41.794811309,-87.652466989,"(41.794811309, -87.652466989)"
5679863,HN488302,07/24/2007 01:00:00 PM,082XX S TALMAN AVE,0460,BATTERY,SIMPLE,STREET,false,false,0835,008,18,70,08B,1160134,1850078,2007,04/15/2016 08:55:02 AM,41.744314668,-87.688830696,"(41.744314668, -87.688830696)"
5679864,HN487195,07/24/2007 02:20:00 PM,034XX N MILWAUKEE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,false,true,1731,017,30,16,08B,1148204,1922490,2007,04/15/2016 08:55:02 AM,41.943259605,-87.730682304,"(41.943259605, -87.730682304)"
5679865,HN484199,07/21/2007 12:30:00 PM,035XX W BELMONT AVE,0820,THEFT,$500 AND UNDER,STREET,false,f

cat: Unable to write to output stream.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = (SparkSession
         .builder
         .config('spark.ui.port', '0')
         .appName('Exercise1-GetMonthlyCrimeCountByType')
         .master('yarn')
         .getOrCreate()
        )

In [4]:
spark

In [46]:
crime_df = (spark
            .read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv("/public/crime/csv/crime_data.csv")
).withColumn("Month", concat(substring('Date',7,4),substring('Date',1,2)).cast("int")).select("Month", "Primary Type")

crime_df.show()

+------+-----------------+
| Month|     Primary Type|
+------+-----------------+
|200707|  CRIMINAL DAMAGE|
|200707|          BATTERY|
|200707|          BATTERY|
|200707|            THEFT|
|200707|  CRIMINAL DAMAGE|
|200707|  CRIMINAL DAMAGE|
|200707|            THEFT|
|200707|            THEFT|
|200707|WEAPONS VIOLATION|
|200707|         BURGLARY|
|200707|            THEFT|
|200707|          BATTERY|
|200707|            THEFT|
|200704|    OTHER OFFENSE|
|200707|            THEFT|
|200707|            THEFT|
|200707|          BATTERY|
|200707|        NARCOTICS|
|200707|          BATTERY|
|200707|    OTHER OFFENSE|
+------+-----------------+
only showing top 20 rows



In [47]:
crimes_grouped_df = (crime_df
                     .groupBy("Primary Type", "Month")
                     .count()
                     .orderBy(["Month", "count"],ascending=[1,0])
                     .select(col("Month"), col("count").alias("crime count"), col("Primary Type").alias("crime type"))
                    )

crimes_grouped_df.show()

+------+-----------+--------------------+
| Month|crime count|          crime type|
+------+-----------+--------------------+
|200101|       7866|               THEFT|
|200101|       6525|             BATTERY|
|200101|       4714|           NARCOTICS|
|200101|       3966|     CRIMINAL DAMAGE|
|200101|       2799|       OTHER OFFENSE|
|200101|       2123|             ASSAULT|
|200101|       2095| MOTOR VEHICLE THEFT|
|200101|       1934|            BURGLARY|
|200101|       1396|             ROBBERY|
|200101|       1387|  DECEPTIVE PRACTICE|
|200101|       1191|   CRIMINAL TRESPASS|
|200101|        563|        PROSTITUTION|
|200101|        337|   WEAPONS VIOLATION|
|200101|        239|OFFENSE INVOLVING...|
|200101|        218|         SEX OFFENSE|
|200101|        162| CRIM SEXUAL ASSAULT|
|200101|        161|PUBLIC PEACE VIOL...|
|200101|        101|LIQUOR LAW VIOLATION|
|200101|         75|          KIDNAPPING|
|200101|         67|               ARSON|
+------+-----------+--------------

In [52]:
(crimes_grouped_df
 .rdd
 .map(lambda x: (str(x[0])+"\t"+str(x[1])+"\t"+str(x[2])))
 .coalesce(1)
 .saveAsTextFile("/user/ranga_rao/solutions/solution01/crimes_by_type_by_month","org.apache.hadoop.io.compress.GzipCodec")
)

In [6]:
crimes_groupedByType_df = (spark
                           .read
                           .option("sep", "\t")
                           .csv("/user/ranga_rao/solutions/solution01/crimes_by_type_by_month/part-00000.gz")
                           
)

crimes_groupedByType_df.show()

+------+--------------------+----+
|   _c0|                 _c1| _c2|
+------+--------------------+----+
|200101|               THEFT|7866|
|200101|             BATTERY|6525|
|200101|           NARCOTICS|4714|
|200101|     CRIMINAL DAMAGE|3966|
|200101|       OTHER OFFENSE|2799|
|200101|             ASSAULT|2123|
|200101| MOTOR VEHICLE THEFT|2095|
|200101|            BURGLARY|1934|
|200101|             ROBBERY|1396|
|200101|  DECEPTIVE PRACTICE|1387|
|200101|   CRIMINAL TRESPASS|1191|
|200101|        PROSTITUTION| 563|
|200101|   WEAPONS VIOLATION| 337|
|200101|OFFENSE INVOLVING...| 239|
|200101|         SEX OFFENSE| 218|
|200101| CRIM SEXUAL ASSAULT| 162|
|200101|PUBLIC PEACE VIOL...| 161|
|200101|LIQUOR LAW VIOLATION| 101|
|200101|          KIDNAPPING|  75|
|200101|               ARSON|  67|
+------+--------------------+----+
only showing top 20 rows

